# Installs

In [ ]:
!pip install psycopg2-binary

In [1]:
# Imports
import psycopg2
import pandas as pd
import string
import json
import re

# Ingest

In [2]:
with open('twitter-nodes-company.jl', 'r') as json_file:
    json_list = json_file
    df = pd.read_json(json_list, lines=True)
df

,type,name,url,attributes
0,company,Nestlé,https://www.nestle.com/,"{'founded': '1 1866; 156 years ago', 'numbe..."
1,company,Verizon Communications,https://www.verizon.com/,"{'founded': '1 October 7, 1983; 38 years ag..."
2,company,The Walt Disney Company,https://thewaltdisneycompany.com/,"{'founded': '1 October 16, 1923; 98 years a..."
3,company,Kraft Heinz,https://www.kraftheinzcompany.com/,"{'founded': '1 July 2, 2015; 6 years ago', ..."
4,company,Unilever,https://www.unilever.com/,{'founded': '1 2 September 1929; 92 years a...
...,...,...,...,...
104,company,Smith Micro Software,https://www.smithmicro.com/,"{'founded': '1 1982', 'numberEmployees': '1..."
105,company,Qumu Corporation,https://qumu.com/,"{'founded': '1 2002', 'numberEmployees': None}"
106,company,Worlds Incorporated,https://www.worlds.com/,"{'founded': '', 'numberEmployees': ''}"
107,company,Voip-Pal,https://www.voip-pal.com/,"{'founded': '', 'numberEmployees': ''}"


In [44]:
df.dtypes

type          string
name          string
url           string
attributes    string
dtype: object

In [43]:
# this did not make a difference - needs escape charater if you want to keep punct
df['type'] = df['type'].astype('string')
df['name'] = df['name'].astype('string')
df['url'] = df['url'].astype('string')
df['attributes'] = df['attributes'].astype('string')

In [65]:
with open('twitter-nodes-other.jl', 'r') as json_file:
    json_list = json_file
    other_df = pd.read_json(json_list, lines=True)
other_df

,type,name,description,attributes,url,table,keywords,author,text
0,service,advertising,,NaN,NaN,NaN,NaN,NaN,NaN
1,industry,1 Food processing,NaN,"{'NAICScode': '', 'NAICStitle': '', 'descripti...",NaN,NaN,NaN,NaN,NaN
2,industry,1 Telecommunications Mass media,NaN,"{'NAICScode': '', 'NAICStitle': '', 'descripti...",NaN,NaN,NaN,NaN,NaN
3,industry,1 Mass media Entertainment,NaN,"{'NAICScode': '', 'NAICStitle': '', 'descripti...",NaN,NaN,NaN,NaN,NaN
4,industry,1 Food processing,NaN,"{'NAICScode': '', 'NAICStitle': '', 'descripti...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
203,article,Business magnate,NaN,NaN,https://en.wikipedia.org/wiki/Business_magnate,,"[Articles containing Japanese-language text, A...",Wikipedia,A business magnate is a person who has achieve...
204,article,Businessperson,NaN,NaN,https://en.wikipedia.org/wiki/Businessperson,,[All Wikipedia articles written in American En...,Wikipedia,A businessman or businesswoman is an individua...
205,article,ByteDance,NaN,NaN,https://en.wikipedia.org/wiki/ByteDance,,"[AC with 0 elements, Articles containing Chine...",Wikipedia,ByteDance Ltd. (Chinese: 字节跳动; pinyin: Zìjié T...
206,article,CBS Interactive,NaN,NaN,https://en.wikipedia.org/wiki/Paramount_Streaming,,"[1992 establishments in California, American c...",Wikipedia,Paramount Streaming (formerly CBS Digital Medi...


In [74]:
other_df.dtypes

type           object
name           object
description    object
attributes     object
url            object
table          object
keywords       object
author         object
text           object
dtype: object

# Database Connection and Load

In [76]:
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')

In [63]:
#Create new graph 'twitter', execute and commit command

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; SELECT create_graph('twitter_companies');"
cur = connection.cursor()
cur.execute(pgt)
connection.commit()

## nodes

In [64]:
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ CREATE (:Company {url:%s,name:%s}) $$) as (n agtype);"

cur = connection.cursor()
for x in range(0, len(df)):
    name=df.iloc[x][1]
    name=name.translate(str.maketrans('','',string.punctuation))
    url=df.iloc[x][2]
    data= (url,name,)
    try:
        cur.execute(pgt, data)
        connection.commit()
    except Exception as e:
        print(e)
        pass

In [87]:
#articles
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ CREATE (:Article {url:%s,name:%s}) $$) as (n agtype);"

cur = connection.cursor()
for x in range(0, len(other_df)):
    t=other_df.iloc[x][0]
    name=other_df.iloc[x][1]
    name=name.translate(str.maketrans('','',string.punctuation))
    url=other_df.iloc[x][4]
    data= (url,name,)
    if t == 'article':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

In [88]:
#services
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ CREATE (:Service {name:%s}) $$) as (n agtype);"

cur = connection.cursor()
for x in range(0, len(other_df)):
    t=other_df.iloc[x][0]
    name=other_df.iloc[x][1]
    name=name.translate(str.maketrans('','',string.punctuation))
    data= (name,)
    if t == 'service':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

In [89]:
#industries
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ CREATE (:Industry {name:%s}) $$) as (n agtype);"

cur = connection.cursor()
for x in range(0, len(other_df)):
    t=other_df.iloc[x][0]
    name=other_df.iloc[x][1]
    name=name.translate(str.maketrans('','',string.punctuation))
    data= (name,)
    if t == 'industry':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

## edges

In [91]:
with open('twitter-edges.jl', 'r') as json_file:
    json_list = json_file
    edge_df = pd.read_json(json_list, lines=True)
edge_df

,company,relation,service,industry,company1,company2,article
0,Twitter,sells,advertising,NaN,NaN,NaN,NaN
1,Nestlé,is_in,NaN,1 Food processing,NaN,NaN,NaN
2,Nestlé,buys,advertising,NaN,NaN,NaN,NaN
3,Verizon Communications,is_in,NaN,1 Telecommunications Mass media,NaN,NaN,NaN
4,Verizon Communications,buys,advertising,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
302,Twitter,has_article,NaN,NaN,NaN,NaN,Business magnate
303,Twitter,has_article,NaN,NaN,NaN,NaN,Businessperson
304,Twitter,has_article,NaN,NaN,NaN,NaN,ByteDance
305,Twitter,has_article,NaN,NaN,NaN,NaN,CBS Interactive


In [113]:
edge_df.dtypes

company     string
relation    string
service     string
industry    string
company1    string
company2    string
article     string
dtype: object

In [95]:
company=edge_df.iloc[x][0]
company

nan

In [112]:
edge_df['company'] = edge_df['company'].astype('string')
edge_df['relation'] = edge_df['relation'].astype('string')
edge_df['service'] = edge_df['service'].astype('string')
edge_df['industry'] = edge_df['industry'].astype('string')
edge_df['company1'] = edge_df['company1'].astype('string')
edge_df['company2'] = edge_df['company2'].astype('string')
edge_df['article'] = edge_df['article'].astype('string')

In [ ]:
#Create a new edge called 'ISIN' to link names of actors to the movie titles
#Note Load time may take a while

cur = connection.cursor()
for x in range(0, len(edge_df)):
    company=edge_df.iloc[x][0]
    company=company.translate(str.maketrans('','',string.punctuation))
    data= (company, tconst,)
    try:
        pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
        SELECT * FROM cypher('twitter_companies', $$ MATCH (a:Company), (b:Service) WHERE a.name='Twitter' \
        AND b.name=advertising CREATE (a)-[e:sells]->(b) RETURN e $$) as (e agtype);"
        cur.execute(pgt, data)
        connection.commit()
    except Exception as e:
        print(e)
        pass

In [114]:
#sells
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ MATCH (a:Company), (b:Service) WHERE a.name=%s \
        AND b.name=%s CREATE (a)-[e:SELLS]->(b) RETURN e $$) as (e agtype);"

cur = connection.cursor()
for x in range(0, len(edge_df)):
    t=edge_df.iloc[x][1]
    company=edge_df.iloc[x]['company']
    print(company)
    print(type(company))
    if type(company) == 'str':
        company=company.translate(str.maketrans('','',string.punctuation))
    service=edge_df.iloc[x][2]
    data= (company,service,)
    if t == 'sells':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

Twitter
<class 'str'>
Nestlé
<class 'str'>
Nestlé
<class 'str'>
Verizon Communications
<class 'str'>
Verizon Communications
<class 'str'>
The Walt Disney Company
<class 'str'>
The Walt Disney Company
<class 'str'>
Kraft Heinz
<class 'str'>
Kraft Heinz
<class 'str'>
Unilever
<class 'str'>
Unilever
<class 'str'>
Kellogg's
<class 'str'>
Kellogg's
<class 'str'>
GlaxoSmithKline
<class 'str'>
GlaxoSmithKline
<class 'str'>
The Penny Hoarder
<class 'str'>
Comcast
<class 'str'>
Comcast
<class 'str'>
CVS Pharmacy
<class 'str'>
CVS Pharmacy
<class 'str'>
Firebelly Marketing
<class 'str'>
Social Driver
<class 'str'>
Chatter Kick
<class 'str'>
Thrive Agency
<class 'str'>
Shoot You
<class 'str'>
eDesign Interactive
<class 'str'>
Lyfe Marketing
<class 'str'>
Seoplus
<class 'str'>
Visual Fizz
<class 'str'>
Metric Theory
<class 'str'>
Oyova
<class 'str'>
Bastion Elevate
<class 'str'>
Codedesign
<class 'str'>
Chamber Media
<class 'str'>
Sachs Marketing Group
<class 'str'>
Coal Marketing
<class 'str'>
Br

In [124]:
#is_in
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ MATCH (a:Company), (b:Industry) WHERE a.name=%s \
        AND b.name=%s CREATE (a)-[e:ISIN]->(b) RETURN e $$) as (e agtype);"

cur = connection.cursor()
for x in range(0, len(edge_df)):
    t=edge_df.iloc[x][1]
    if t == 'is_in':
        company=edge_df.iloc[x]['company']
#         print(company)
#         print(type(company))
        company=company.translate(str.maketrans('','',string.punctuation))
        industry=edge_df.iloc[x]['industry']
        industry=industry.translate(str.maketrans('','',string.punctuation))
        data= (company,industry,)
    if t == 'is_in':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

In [127]:
#has_article
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ MATCH (a:Company), (b:Article) WHERE a.name=%s \
        AND b.name=%s CREATE (a)-[e:HASARTICLE]->(b) RETURN e $$) as (e agtype);"

cur = connection.cursor()
for x in range(0, len(edge_df)):
    t=edge_df.iloc[x][1]
    if t == 'has_article':
        company=edge_df.iloc[x]['company']
#         print(company)
#         print(type(company))
        company=company.translate(str.maketrans('','',string.punctuation))
        article=edge_df.iloc[x]['article']
        article=article.translate(str.maketrans('','',string.punctuation))
        data= (company,article,)
    if t == 'has_article':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

In [130]:
#has_potential_entrant
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ MATCH (a:Company), (b:Company) WHERE a.name=%s \
        AND b.name=%s CREATE (a)-[e:HASPOTENTIALENTRANT]->(b) RETURN e $$) as (e agtype);"

cur = connection.cursor()
for x in range(0, len(edge_df)):
    t=edge_df.iloc[x][1]
    if t == 'has_potential_entrant':
        company1=edge_df.iloc[x]['company1']
#         print(company)
#         print(type(company))
        company1=company1.translate(str.maketrans('','',string.punctuation))
        company2=edge_df.iloc[x]['company2']
        company2=company2.translate(str.maketrans('','',string.punctuation))
        data= (company1,company2,)
    if t == 'has_potential_entrant':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

In [135]:
#has_substitute
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')
    
# other node types
#Note Load time may take a while

pgt="LOAD '$libdir/plugins/age'; SET search_path=ag_catalog; \
SELECT * FROM cypher('twitter_companies', $$ MATCH (a:Company), (b:Company) WHERE a.name=%s \
        AND b.name=%s CREATE (a)-[e:HASSUBSTITUTE]->(b) RETURN e $$) as (e agtype);"

cur = connection.cursor()
for x in range(0, len(edge_df)):
    t=edge_df.iloc[x][1]
    if t == 'has_substitute':
        company1=edge_df.iloc[x]['company1']
#         print(company)
#         print(type(company))
        company1=company1.translate(str.maketrans('','',string.punctuation))
        company2=edge_df.iloc[x]['company2']
        company2=company2.translate(str.maketrans('','',string.punctuation))
        data= (company1,company2,)
    if t == 'has_substitute':
        try:
            cur.execute(pgt, data)
            connection.commit()
        except Exception as e:
            print(e)
            pass
cur.close()
connection.close()

# Check the write

In [136]:
#Establish Connection to Postgres
connection = psycopg2.connect(user='sicklerr',host="192.46.216.236",port="5432", \
                              database='sicklerr', password='Knowledge$43')

cur = connection.cursor()

pgt="LOAD '$libdir/plugins/age'; \
SET search_path=ag_catalog; select * from cypher('twitter_companies', $$ MATCH \
(a)-[r]->(b) RETURN a.name, r, b.name $$) as (name agtype, label agtype, service agtype);"

cur.execute(pgt)
res=cur.fetchall()
print(type(res))

<class 'list'>


In [137]:
res[:1000]

[('"Twitter"',
  '{"id": 1970324836974593, "label": "SELLS", "end_id": 1407374883553281, "start_id": 844424930132049, "properties": {}}::edge',
  '"advertising"'),
 ('"Twitter"',
  '{"id": 1970324836974594, "label": "SELLS", "end_id": 1407374883553281, "start_id": 844424930132049, "properties": {}}::edge',
  '"advertising"'),
 ('"Twitter"',
  '{"id": 1970324836974595, "label": "SELLS", "end_id": 1407374883553281, "start_id": 844424930132049, "properties": {}}::edge',
  '"advertising"'),
 ('"Twitter"',
  '{"id": 1970324836974596, "label": "SELLS", "end_id": 1407374883553281, "start_id": 844424930132049, "properties": {}}::edge',
  '"advertising"'),
 ('"Twitter"',
  '{"id": 1970324836974597, "label": "SELLS", "end_id": 1407374883553281, "start_id": 844424930132049, "properties": {}}::edge',
  '"advertising"'),
 ('"Twitter"',
  '{"id": 1970324836974598, "label": "SELLS", "end_id": 1407374883553281, "start_id": 844424930132049, "properties": {}}::edge',
  '"advertising"'),
 ('"Twitter"',
 

In [120]:
cur.close()
connection.close()